In [1]:
push!(LOAD_PATH, ".")
using BenchmarkTools
using IntervalSets

In [21]:
# Knots
struct Knots
    vector :: Array{Float64,1}
    function Knots(vector)
        new(sort(vector))
    end
end

Base.:+(k₁::Knots, k₂::Knots) = Knots(sort([k₁.vector...,k₂.vector...]))
Base.:*(p₊::Int, k::Knots) = if (p₊==0)
        Knots([])
    elseif (p₊>0)
        sum(k for _ ∈ 1:p₊)
    else
        error("p₊ must be non-negative")
    end
    
Base.in(r::Real, k::Knots) = in(r,k.vector)
Base.getindex(k::Knots, i::Int) = k.vector[i]
Base.getindex(k::Knots, v::AbstractArray{Int64,1}) = Knots(k.vector[v])
Base.length(k::Knots) = length(k.vector)
Base.firstindex(k) = 1
Base.lastindex(k) = length(k)
Base.unique(k::Knots) = Knots(unique(k.vector))

function Base.:⊆(k::Knots, k′::Knots)
    K′=copy(k′.vector)
    for kᵢ ∈ k.vector
        i=findfirst(x->x==kᵢ,K′)
        if(i isa Nothing)
            return false
        end
        deleteat!(K′,i)
    end
    return true
end

In [16]:
# B-Spline Space
struct BSplineSpace
    degree::Int
    knots::Knots
    function BSplineSpace(degree::Int, knots::Knots)
        if (degree < 0)
            error("degree of polynominal must be non-negative")
        else
            new(degree,knots)
        end
    end
end

const 𝒫 = BSplineSpace

function dim(bsplinespace::BSplineSpace)
    p=bsplinespace.degree
    k=bsplinespace.knots
    return length(k)-p-1
end

function Base.:⊆(P::BSplineSpace, P′::BSplineSpace)
    p=P.degree
    k=P.knots
    p′=P′.degree
    k′=P′.knots
    p₊=p′-p
    
    return (k+p₊*unique(k) ⊆ k′) && p₊ ≥ 0
end

function Base.iszero(P::BSplineSpace)
    p=P.degree
    k=P.knots
    n=dim(P)
    return [k[i]==k[i+p+1] for i ∈ 1:n]
end

In [22]:
# B-Spline functions
function BSplineBasis(P::BSplineSpace, t)::Array{Float64,1}
    p=P.degree
    k=P.knots
    
    n=length(k)-p-1
    if(p==0)
        @inbounds return [k[i]≤t<k[i+1]||(k[i]≠k[i+1]==k[end]==t) for i ∈ 1:n]
    else
        @inbounds K=[ifelse(k[i+p]==k[i],0,(t-k[i])/(k[i+p]-k[i])) for i ∈ 1:n+1]
        @inbounds B=BSplineBasis(𝒫(p-1,k),t)
        @inbounds return [K[i]*B[i]+(1-K[i+1])*B[i+1] for i ∈ 1:n]
    end
end

function BSplineBasis′(P::BSplineSpace, t)::Array{Float64,1}
    p=P.degree
    k=P.knots

    n=length(k)-p-1
    if(p==0)
        return [0.0 for _ ∈ 1:n]
    else
        @inbounds K=[ifelse(k[i+p]==k[i],0,p/(k[i+p]-k[i])) for i ∈ 1:n+1]
        B=BSplineBasis(𝒫(p-1,k),t)
        @inbounds return [K[i]*B[i]-K[i+1]*B[i+1] for i ∈ 1:n]
    end
end

function BSplineSupport(i::Int64, P::BSplineSpace)::ClosedInterval
    p=P.degree
    k=P.knots

    return k[i]..k[i+p+1]
end

function BSplineCoefficient(P::BSplineSpace, P′::BSplineSpace)::Array{Float64,2}
    p=P.degree
    k=P.knots
    p′=P′.degree
    k′=P′.knots

    p₊=p′-p
    if(P ⊆ P′)
        if(p == 0 && p' ≥ 0)
            n=length(k)-1
            n′=length(k′)-p₊-1
            A⁰=Float64[BSplineSupport(j,𝒫(p₊,k′)) ⊆ BSplineSupport(i,𝒫(0,k)) for i ∈ 1:n, j ∈ 1:n′]
            return A⁰
        else
            Aᵖ⁻¹=BSplineCoefficient(𝒫(p-1, k), 𝒫(p′-1, k′))
            n=length(k)-p-1
            n′=length(k′)-p′-1
            K′=[k′[i+p′]-k′[i] for i ∈ 1:n′+1]
            K=[1/(k[i+p]-k[i]) for i ∈ 1:n+1]
            LS=(p/p′)*[K′[j]*(K[i]*Aᵖ⁻¹[i,j]-K[i+1]*Aᵖ⁻¹[i+1,j]) for i ∈ 1:n, j ∈ 1:n′+1]
            Aᵖ₊, Aᵖ₋=zeros(n,n′), zeros(n,n′)
            Aᵖ₊[:,1]=LS[:,1]
            Aᵖ₋[:,n′]=-LS[:,n′+1]
            for j ∈ 2:n′
                Aᵖ₊[:,j]+=Aᵖ₊[:,j-1]+LS[:,j]
                Aᵖ₋[:,(n′-j+2)-1]+=Aᵖ₋[:,(n′-j+2)]-LS[:,(n′-j+2)]
            end
            return (Aᵖ₊+Aᵖ₋).*Float64[BSplineSupport(j,𝒫(p′,k′)) ⊆ BSplineSupport(i,𝒫(p,k)) for i ∈ 1:n, j ∈ 1:n′]/2
        end
    else
        error("𝒫[p,k] ⊄ 𝒫[p′,k′]")
    end
end

BSplineCoefficient (generic function with 1 method)

In [41]:
function BSplineCoefficient5(P::BSplineSpace, P′::BSplineSpace)::Array{Float64,2}
    p=P.degree
    k=P.knots
    p′=P′.degree
    k′=P′.knots

    p₊=p′-p
    if(P ⊆ P′)
        if(p == 0 && p' ≥ 0)
            n=length(k)-1
            n′=length(k′)-p₊-1
            A⁰=Float64[BSplineSupport(j,𝒫(p₊,k′)) ⊆ BSplineSupport(i,𝒫(0,k)) for i ∈ 1:n, j ∈ 1:n′]
            A⁰[:,findall(iszero(P′))].=NaN
            return A⁰
        else
            Aᵖ⁻¹=BSplineCoefficient5(𝒫(p-1, k), 𝒫(p′-1, k′))
            n=length(k)-p-1
            n′=length(k′)-p′-1
            K′=[k′[i+p′]-k′[i] for i ∈ 1:n′+1]
            K=[1/(k[i+p]-k[i]) for i ∈ 1:n+1]
            LS=(p/p′)*[K′[j]*(K[i]*Aᵖ⁻¹[i,j]-K[i+1]*Aᵖ⁻¹[i+1,j]) for i ∈ 1:n, j ∈ 1:n′+1]
            Aᵖ₊, Aᵖ₋=zeros(n,n′), zeros(n,n′)
            Aᵖ₊[:,1]=LS[:,1]
            Aᵖ₋[:,n′]=-LS[:,n′+1]
            for j ∈ 2:n′
                Aᵖ₊[:,j]+=Aᵖ₊[:,j-1]+LS[:,j]
                Aᵖ₋[:,(n′-j+2)-1]+=Aᵖ₋[:,(n′-j+2)]-LS[:,(n′-j+2)]
            end
            return (Aᵖ₊+Aᵖ₋).*Float64[BSplineSupport(j,𝒫(p′,k′)) ⊆ BSplineSupport(i,𝒫(p,k)) for i ∈ 1:n, j ∈ 1:n′]/2
        end
    else
        error("𝒫[p,k] ⊄ 𝒫[p′,k′]")
    end
end

BSplineCoefficient5 (generic function with 1 method)

In [ ]:
zeros()

In [49]:
P=𝒫(1,Knots([1,2,3,4,5]))
P′=𝒫(2,2Knots([1,2,3,4,5]))
BSplineCoefficient5(P,P′)

3×7 Array{Float64,2}:
 0.5  1.0  0.5  0.0  0.0  0.0  0.0
 0.0  0.0  0.5  1.0  0.5  0.0  0.0
 0.0  0.0  0.0  0.0  0.5  1.0  0.5

In [35]:
A=zeros(3,5)

3×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [292]:
typeof(NaN)

Float64

In [378]:
p=1
k=Knots([1,4,5,5,8,10])
p′=1
k′=k+Knots(rand(1:10,3))

P=𝒫(p,k)
P′=𝒫(p′,k′)

if(P ⊆ P′)
    println("𝒫[p,k] ⊆ 𝒫[p′,k′]")
    
    n=dim(𝒫(p,k))
    n′=dim(𝒫(p′,k′))
    
    Z′ᵖ⁻¹=iszero(𝒫(p′-1,k′))
    W=findall(Z′ᵖ⁻¹)
    Aᵖ⁻¹=rand(n+1,n′+1)
    Aᵖ⁻¹[:,W].=NaN

    Aᵖ=zeros(n,n′)
    Aᵖ[:,1].=5
    Aᵖ[:,end].=5
    Z′ᵖ=iszero(𝒫(p′,k′))
    
    Q=[1:W[1]-1,[W[i]:W[i+1]-1 for i ∈ 1:length(W)-1]...,W[end]:n′]
    l=length(Q)
    L=length.(Q)
    Ãᵖ=[Aᵖ[:,q] for q ∈ Q]
    for i ∈ 2:l-1
        if(L[i]==1)
            Ãᵖ[i].=NaN
        end
    end
    for i ∈ 1:l-1
        if(L[i] ≥ 2)
            Ãᵖ[i][:,end].=1.0
        end
    end
    for i ∈ 2:l
        if(L[i] ≥ 2)
            Ãᵖ[i][:,1].=-1.0
        end
    end
    for i ∈ 1:l
        if(L[i] ≥ 3)
            A₊=copy(Ãᵖ[i])
            A₋=copy(Ãᵖ[i])
            for j ∈ 2:L[i]-1
                Ãᵖ[i][:,j].=-3
            end
        end
    end
    Aᵖ=hcat(Ãᵖ...)
end

𝒫[p,k] ⊆ 𝒫[p′,k′]


4×7 Array{Float64,2}:
 -1.0  -3.0  1.0  NaN  -1.0  -3.0  5.0
 -1.0  -3.0  1.0  NaN  -1.0  -3.0  5.0
 -1.0  -3.0  1.0  NaN  -1.0  -3.0  5.0
 -1.0  -3.0  1.0  NaN  -1.0  -3.0  5.0

In [379]:
Aᵖ⁻¹

5×8 Array{Float64,2}:
 NaN  0.0465132  0.911305   NaN  NaN  0.122476   0.353256  0.843464 
 NaN  0.545535   0.0961825  NaN  NaN  0.651643   0.539679  0.811087 
 NaN  0.0956857  0.0164693  NaN  NaN  0.378976   0.885324  0.0228238
 NaN  0.994331   0.713917   NaN  NaN  0.973353   0.439295  0.709257 
 NaN  0.203944   0.435752   NaN  NaN  0.0369813  0.627456  0.0547634

In [369]:
Aᵖ

4×7 Array{Float64,2}:
 -1.0  -3.0  1.0  NaN  NaN  -1.0  5.0
 -1.0  -3.0  1.0  NaN  NaN  -1.0  5.0
 -1.0  -3.0  1.0  NaN  NaN  -1.0  5.0
 -1.0  -3.0  1.0  NaN  NaN  -1.0  5.0

In [358]:
rand(3)

3-element Array{Float64,1}:
 0.4750840893066895 
 2.7137575731077015 
 0.13764435351378468

In [360]:
rand(1:10,3)

3-element Array{Int64,1}:
 7
 6
 7

In [381]:
[1:5;]

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [384]:
(1:5)

1:5

In [388]:
;

ArgumentError: ArgumentError: no cmd to execute

In [387]:
(hogea;)

UndefVarError: UndefVarError: hogea not defined

In [299]:
length.(Q)

4-element Array{Int64,1}:
 4
 1
 2
 0

In [163]:
m=5
Z′ᵖ=[3,5,6]

3-element Array{Int64,1}:
 3
 5
 6

4-element Array{UnitRange{Int64},1}:
 1:2
 3:4
 5:5
 6:5

4-element Array{Array{Float64,2},1}:
 [5.0 0.0; 5.0 0.0; 5.0 0.0]
 [0.0 0.0; 0.0 0.0; 0.0 0.0]
 [0.0; 0.0; 0.0]            
 Array{Float64}(undef,3,0)  

In [160]:
hoge[3]

2×1 Array{Float64,2}:
 0.12717542888207234
 0.02708144613825647

In [146]:
[2,3,4][1:0]

0-element Array{Int64,1}

In [67]:
Base.showarray(STDOUT,rand(3,3),false)

UndefVarError: UndefVarError: showarray not defined

In [34]:
A

3×5 Array{Float64,2}:
 0.0  NaN  NaN  0.0  0.0
 0.0  NaN  NaN  0.0  0.0
 0.0  NaN  NaN  0.0  0.0

In [28]:
A

3×5 Array{Float64,2}:
 0.0  3.0  0.0  0.0  0.0
 0.0  3.0  0.0  0.0  0.0
 0.0  3.0  0.0  0.0  0.0

In [32]:
findall([true, false,true])

2-element Array{Int64,1}:
 1
 3

In [18]:
struct BSplineManifold
    bsplinespaces::Array{BSplineSpace,1}
    controlpoints::Array{Float64}
    function BSplineManifold(bsplinespaces::Array{BSplineSpace,1}, controlpoints::Array{Float64})
        if (collect(size(controlpoints)[1:end-1]) ≠ dim.(bsplinespaces))
            error("dimension does not match")
        else
            new(bsplinespaces, controlpoints)
        end
    end
end

function BSplineBasis(𝒫s::Array{BSplineSpace,1},t)
    if(length(𝒫s)==length(t)==1)
        return BSplineBasis(𝒫s[1],t[1])
    elseif(length(𝒫s)==length(t)==2)
        return BSplineBasis(𝒫s[1],t[1])*BSplineBasis(𝒫s[2],t[2])'
    else
        error("dimension does not match")
    end
end

function Mapping(M::BSplineManifold, t::Array{Float64,1})
    𝒫s = M.bsplinespaces
    𝒂 = M.controlpoints
    d=length(𝒫s)
    d̂=size(𝒂)[end]
    return [sum(BSplineBasis(𝒫s,t).*𝒂[:,:,i]) for i ∈ 1:d̂]
end

Mapping (generic function with 1 method)

In [19]:
function Refinement(M::BSplineManifold, 𝒫s′::Array{BSplineSpace,1})
    𝒫s = M.bsplinespaces
    𝒂 = M.controlpoints
    d̂ = size(𝒂)[end]
    n = dim.(𝒫s)
    n′ = dim.(𝒫s′)
    if(prod(𝒫s .⊆ 𝒫s′))
        A = BSplineCoefficient.(𝒫s,𝒫s′)
        global XX=A
        𝒂′ = [sum(A[1][I₁,J₁]*A[2][I₂,J₂]*𝒂[I₁,I₂,i] for I₁ ∈ 1:n[1], I₂ ∈ 1:n[2]) for J₁ ∈ 1:n′[1], J₂ ∈ 1:n′[2], i ∈ 1:d̂]
        return BSplineManifold(𝒫s′, 𝒂′)
    else
        error("𝒫[p,k] ⊄ 𝒫[p′,k′]")
    end
end

Refinement (generic function with 1 method)

In [9]:
k₁=k₂=Knots([-0.1,0,1,1.1])
p₁=p₂=1

𝒂=Float64[ifelse(i==1, 2*I₁, 3*I₂) for I₁ ∈ 0:1,  I₂ ∈ 0:1, i ∈ 1:2]
𝒫s=[𝒫(p₁,k₁), 𝒫(p₂,k₂)]
M=BSplineManifold(𝒫s,𝒂)

t=[0.2,0.3]
Mapping(M,t)

2-element Array{Float64,1}:
 0.39999999999999997
 0.8999999999999999 

In [10]:
p′₁=p′₂=3
k′₁=k₁+2unique(k₁)+Knots(rand(300))
k′₂=k₂+2unique(k₂)+Knots(rand(300))
M′=Refinement(M, [𝒫(p′₁,k′₁), 𝒫(p′₂,k′₂)])

BSplineManifold(BSplineSpace[BSplineSpace(3, Knots([-0.1, -0.1, -0.1, 0.0, 0.0, 0.0, 0.00018114447440020776, 0.005682147262958459, 0.007519778511360542, 0.013767617807010302  …  0.9771439698090276, 0.9807708833570186, 0.9952747181171913, 0.9989982156591961, 1.0, 1.0, 1.0, 1.1, 1.1, 1.1])), BSplineSpace(3, Knots([-0.1, -0.1, -0.1, 0.0, 0.0, 0.0, 0.0029830994029973468, 0.005230329096788777, 0.008620617906021089, 0.010137138243494714  …  0.9810409562255265, 0.9886151016816482, 0.9947090267408802, 0.998880124916182, 1.0, 1.0, 1.0, 1.1, 1.1, 1.1]))], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.444444444444445 0.8888888888888897 … 0.8888888888888891 0.4444444444444444; 0.22222222222222257 0.444444444444445 … 0.4444444444444447 0.22222222222222227]

[0.0 0.0 … 0.6666666666666671 0.33333333333333337; 0.0 0.0 … 1.3333333333333335 0.6666666666666666; … ; 0.0 0.0 … 1.3333333333333335 0.6666666666666666; 0.0 0.0 … 0.6666666666666671 0.33333333333333337])

In [11]:
t=rand(2)
@benchmark Mapping(M′,t)

BenchmarkTools.Trial: 
  memory estimate:  4.66 MiB
  allocs estimate:  14979
  --------------
  minimum time:     645.087 μs (0.00% GC)
  median time:      723.404 μs (0.00% GC)
  mean time:        891.210 μs (12.82% GC)
  maximum time:     4.528 ms (31.89% GC)
  --------------
  samples:          5562
  evals/sample:     1

In [66]:
@benchmark Mapping(M,t)

BenchmarkTools.Trial: 
  memory estimate:  5.00 KiB
  allocs estimate:  121
  --------------
  minimum time:     5.430 μs (0.00% GC)
  median time:      5.808 μs (0.00% GC)
  mean time:        6.537 μs (8.41% GC)
  maximum time:     946.530 μs (98.35% GC)
  --------------
  samples:          10000
  evals/sample:     6

In [13]:
p=4
p′=17
p₊=p′-p
k=Knots((9*rand(20)).+3)
k₊=Knots(((k[end]-k[1])*rand(13)).+k[1])
k′=k+p₊*unique(k)+k₊

Knots([3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397  …  11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171])

In [14]:
@benchmark BSplineBasis(𝒫(p,k),6.3)  # ちょっと遅いのはノット列とかを毎度型検査(?)してるから?

BenchmarkTools.Trial: 
  memory estimate:  3.03 KiB
  allocs estimate:  41
  --------------
  minimum time:     2.005 μs (0.00% GC)
  median time:      2.148 μs (0.00% GC)
  mean time:        2.508 μs (8.17% GC)
  maximum time:     509.071 μs (98.40% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [15]:
BSplineSpace(3,Knots([1,2,3]))

BSplineSpace(3, Knots([1.0, 2.0, 3.0]))

In [16]:
A=BSplineCoefficient(𝒫(p,k), 𝒫(p′,k′))

15×275 Array{Float64,2}:
  1.81166e-5   0.000160061   0.000701802  …  -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0              0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
 -0.0         -0.0          -0.0          …   0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0          …  -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
  0.0          0.0           0.0             -0.0   

In [17]:
𝒫(p,k) ⊆ 𝒫(p′,k′)

true

In [21]:
1.0 ∈ Knots([1])

true

In [20]:
Base.in(r::Real, k::Knots) = in(r,k.vector)

In [22]:
missing

missing

In [24]:
null

UndefVarError: UndefVarError: null not defined

In [23]:
Araryaundef

array initializer with undefined values

In [32]:
hoge=Array{Float64}(undef, 3,3)

3×3 Array{Float64,2}:
 3.8382e151   4.50622e-144  1.41529e161
 1.47722e179  3.80985e180   6.00737e-67
 8.37404e242  1.06396e224   3.88664e97 

In [34]:
hoge[1,1]=nothing

MethodError: MethodError: Cannot `convert` an object of type Nothing to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [31]:
nothing

In [36]:
?empty

search: empty empty! isempty



```
empty(x::Tuple)
```

Returns an empty tuple, `()`.

---

```
empty(v::AbstractVector, [eltype])
```

Create an empty vector similar to `v`, optionally changing the `eltype`.

# Examples

```jldoctest
julia> empty([1.0, 2.0, 3.0])
0-element Array{Float64,1}

julia> empty([1.0, 2.0, 3.0], String)
0-element Array{String,1}
```

---

```
empty(a::AbstractDict, [index_type=keytype(a)], [value_type=valtype(a)])
```

Create an empty `AbstractDict` container which can accept indices of type `index_type` and values of type `value_type`. The second and third arguments are optional and default to the input's `keytype` and `valtype`, respectively. (If only one of the two types is specified, it is assumed to be the `value_type`, and the `index_type` we default to `keytype(a)`).

Custom `AbstractDict` subtypes may choose which specific dictionary type is best suited to return for the given index and value types, by specializing on the three-argument signature. The default is to return an empty `Dict`.


In [39]:
EmptySet

UndefVarError: UndefVarError: EmptySet not defined

In [38]:
emptyset

UndefVarError: UndefVarError: emptyset not defined

In [41]:
1 ∈ Set([1,2,3])

true

In [43]:
Set([1,2,3,1])

Set([2, 3, 1])

In [1]:
Set([1,2,3])

Set([2, 3, 1])